# Notebook for extracting volume sections from entire brain volumes

### Set up

In [1]:
# # Install the required packages
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install opencv-python

## Center coordinates of desired extracted slices
# x, y, z = 2469, 3948, 1011 # Occipital lobe
#x, y, z = 2019, 3956, 3701 # Parietal lobe
#x, y, z = 3049, 4178, 5485 # Frontal_lobe 1
#x, y, z = 3166, 1398, 2443 # Occipital lobe 2
x, y, z = 3868, 3914, 5800 # Frontal lobe 3


import os
import numpy as np
import cv2

# Define image directories

# Parent directory
path = "D:\\Eric"
#path = "C:\\Users\\homeuser\\Documents\\brain"
#path = "D:\\Eric_Brain\\extracted_slices"


# Directory with the images
data_path = os.path.join(path, 'data')

# List of all files in the directory that end with .jp2
files = os.listdir(data_path)
files = [f for f in files if f.endswith('.jp2')]


# Read the first image
dummy_image = cv2.imread(os.path.join(data_path, files[0]), cv2.IMREAD_UNCHANGED)

# Define the dimensions of the desired extracted volume
n = 10 # Power of 2
dx, dy, dz = 2**n, 2**n, 2**n
# Dimensions of the extracted volume
dx_dy_dz = (dx, dy, dz)

# Define the center of the extracted slice
#x, y, z = 3067, 1542, 5118
xyz_center = (x, y, z)

# Shape of the memory-mapped array to be created **
# Going through all the slices is expensive, so we just take the slices we need
shape = (dummy_image.shape[0], dummy_image.shape[1], dz)


# Create the memory-mapped array
name_of_saved_mmap = f"memmap_{x}_{y}_{z}.npy"
mmap = np.memmap(os.path.join(path, name_of_saved_mmap), dtype = dummy_image.dtype, mode = 'w+', shape = shape)
print(f"Is this what you expected? {mmap.shape}")




Is this what you expected? (6448, 6448, 1024)


In [3]:
# Create a function load slices and extract a volume
def load_slices_extract_volume(mmap, name_of_saved_mmap, data_path, files, xyz_center, dx_dy_dz, verbose = True):
    """
    Function to load slices into memory, copy them into the memory mapped array and extract a volume from the memory mapped array.

    Parameters
    ----------
    mmap : numpy.memmap
        The memory mapped that has been created.
    name_of_saved_mmap : str
        The name of the saved memory mapped array.
    data_path : str
        The path to the directory with the images.
    files : list
        List of all files in the directory that end with .jp2.
    xyz_center : tuple
        The center of the extracted volume.
    dx_dy_dz : tuple
        The dimensions of the extracted volume.
    verbose : bool, optional
        If True, print the progress. The default is True.

    Returns
    -------
    Volume: Extracted volume
    """


    # Extract the center of the extracted volume
    x = xyz_center[0]
    y = xyz_center[1]
    z = xyz_center[2]

    # Extract the dimensions of the extracted volume
    dx = dx_dy_dz[0]
    dy = dx_dy_dz[1]
    dz = dx_dy_dz[2]

    # Define a counter to print the progress
    count = np.log(mmap.shape[2]) // np.log(10) - 1
    
    # Load the slices into memory and copy them into the memory mapped array
    for i in range(z-dz//2, z+dz//2):
        # Read the slice into memory, cv2.IMREAD_UNCHANGED is used to read the image as is without any conversion
        slice_data = cv2.imread(os.path.join(data_path, files[i]), cv2.IMREAD_UNCHANGED)
        # Copy the slice into the memory mapped array
        mmap[:, :, i - (z-dz//2)] = slice_data

        if verbose:
            if (i - (z-dz//2)) % 10**count == 0:
                print(f"slice extracted: {i}, slice saved: {i - (z-dz//2)}")

    # Close the memory mapped array**
    del mmap

    # Read the loaded memory mapped array
    mmap = np.memmap(os.path.join(data_path, os.pardir, name_of_saved_mmap), dtype = np.uint16, mode = 'r', shape = shape)
    
    # Extract a volume from the memory mapped array
    volume = mmap[x-dx//2:x+dx//2, y-dy//2:y+dy//2, :]

    # Check the shape of the extracted slice and the data type of the read slice
    print(f"\nExtracted volume shape: {volume.shape}\n")

    # Save the slices as separate JPEG 2000 files
    from pathlib import Path
    extractedv_path = Path(os.path.join(data_path, os.pardir, f"Extracted_slices_x_y_z_{x}_{y}_{z}_shape_{dx}_{dy}_{dz}"))
    # Create the folder if it does not exist
    extractedv_path.mkdir(parents=True, exist_ok=True)
    for s in range(volume.shape[2]):
    # In CV2 16-bit unsigned images can be saved in the case of PNG, JPEG 2000, and TIFF formats.
    # Others will be converted to 8-bit unsigned images and saved that way. 
    # See: https://docs.opencv.org/3.4/d4/da8/group__imgcodecs.html
        filepath = extractedv_path / files[s]
        cv2.imwrite(str(filepath), volume[:, :, s].astype(np.uint16), [cv2.IMWRITE_JPEG2000_COMPRESSION_X1000, 1000])
        # A higher value means higher quality but larger file size

        # Print the slice number saved if verbose is True
        if verbose:
            if s % 10**count == 0:
                print(f"slice saved: {s}")
               
    # Close the memory mapped array**
    del mmap

    return volume
   

In [4]:
# Call the function
extracted_volume = load_slices_extract_volume(mmap, name_of_saved_mmap, data_path, files, xyz_center, dx_dy_dz, verbose = True)

slice extracted: 5288, slice saved: 0
slice extracted: 5388, slice saved: 100
slice extracted: 5488, slice saved: 200
slice extracted: 5588, slice saved: 300
slice extracted: 5688, slice saved: 400
slice extracted: 5788, slice saved: 500
slice extracted: 5888, slice saved: 600
slice extracted: 5988, slice saved: 700
slice extracted: 6088, slice saved: 800
slice extracted: 6188, slice saved: 900
slice extracted: 6288, slice saved: 1000

Extracted volume shape: (1024, 1024, 1024)

slice saved: 0
slice saved: 100
slice saved: 200
slice saved: 300
slice saved: 400
slice saved: 500
slice saved: 600
slice saved: 700
slice saved: 800
slice saved: 900
slice saved: 1000


In [17]:
extracted_volume.shape

(1024, 1024, 1024)

## Appendix

In [ ]:
# count = np.log(shape[2]) // np.log(10) - 1
# # Load the slices into memory and copy them into the memory mapped array
# for i in range(z-dz//2, z+dz//2):
#     # Read the slice into memory, cv2.IMREAD_UNCHANGED is used to read the image as is without any conversion
#     slice_data = cv2.imread(os.path.join(data_path, files[i]), cv2.IMREAD_UNCHANGED)
#     # Copy the slice into the memory mapped array
#     mmap[:, :, i - (z-dz//2)] = slice_data

#     if (i - (z-dz//2)) % 10**count == 0:
#         print(f"slice extracted: {i}, slice saved: {i - (z-dz//2)}")

# # Extract a volume from the memory mapped array
# volume = mmap[x-dx//2:x+dx//2, y-dy//2:y+dy//2, z-dz//2:z+dz//2]

# # Check the shape of the extracted slice and the data type of the read slice
# print(f"Extracted volume shape: {volume.shape}")


# # Save the slices as separate JPEG 2000 files
# from pathlib import Path
# extractedv_path = Path(os.path.join(path, f"Extracted_slices_x_y_z_{x}_{y}_{z}_shape_{dx}_{dy}_{dz}"))
# # Create the folder if it does not exist
# extractedv_path.mkdir(parents=True, exist_ok=True)
# for s in range(volume.shape[2]):
#    # In CV2 16-bit unsigned images can be saved in the case of PNG, JPEG 2000, and TIFF formats.
#    # Others will be converted to 8-bit unsigned images and saved that way. 
#    # See: https://docs.opencv.org/3.4/d4/da8/group__imgcodecs.html
#     filepath = extractedv_path / files[s]
#     cv2.imwrite(str(filepath), volume[:, :, s].astype(np.uint16), [cv2.IMWRITE_JPEG2000_COMPRESSION_X1000, 1000])
#     # A higher value means higher quality but larger file size


# # Close the memory mapped array**
# del mmap